In [1]:
import requests
import os

API_URL = "https://vi.wikipedia.org/w/api.php"

data_file = r'D:\Huan\Project\KHDL\data'
root_category = 'Thể loại:Y học'

dict = set()

#Hàm lấy danh sách các trang và danh sách các thể loại
def extract_pages(category_name):
    params = {
        "action": "query",
        "list": "categorymembers",
        "cmtitle": category_name,
        "cmtype": "page|subcat",
        "cmlimit": "max",
        "format": "json"
    }
    
    #gửi yêu cầu get đến api
    response = requests.get(API_URL, params = params)
    
    #Phân tích kết quả trả về từ api
    data = response.json()
    
    # Lấy danh sách các thành viên trong thể loại
    category_members = data['query']['categorymembers']

    # Phân loại thành thể loại con và trang
    subcategories = []
    pages = []

    for member in category_members:
        if member['title'].startswith("Thể loại:"):
            subcategories.append({'title': member['title'], 'pageid': member['pageid']})
        else:
            pages.append({'title': member['title'], 'pageid': member['pageid']})
     
    return {"subcategories": subcategories, "pages": pages }

# Hàm lấy nội dung của một trang
def get_page_content(page_id):
    params = {
        "action": "query",
        "prop": "extracts",
        "explaintext": True,  # Chỉ lấy nội dung dạng text, không lấy HTML
        "pageids": page_id,   # Lấy nội dung theo page ID
        "format": "json"
    }

    response = requests.get(API_URL, params=params)
    data = response.json()

    pages = data.get('query', {}).get('pages', {})
    for page_id, page in pages.items():
        return page.get('extract', '')

#Hàm lấy nội dung trong các trang của 1 thể loại
def get_all_content(category_name, category_id, dept, max_dept):
    if dept>max_dept:
        return
    if category_id in dict:
        return
    dict.add(category_id)
    
    ep = extract_pages(category_name)
    pages = ep['pages']
    subcategories = ep['subcategories']

    url_folder = data_file + '\\' + str(category_id)
        
    # Tạo thư mục để lưu file nếu chưa có
    if not os.path.exists(url_folder):
        os.makedirs(url_folder)
    os.chdir(url_folder) 
    if len(pages) > 0:
        for page in pages:
            with open(url_folder + '\\' + str(page['pageid']) + '.txt', 'w', encoding = 'utf-8') as f:
                f.write(get_page_content(page['pageid']))
            with open(r'D:\Huan\Project\KHDL\structure.txt', 'a', encoding = 'utf-8') as f:
                f.write('\t' * (dept + 1) + str(page['pageid']) + '//' + page['title'] + '\n')
    
    if len(subcategories) > 0:
        for subcategorie in subcategories:
            with open(r'D:\Huan\Project\KHDL\structure.txt', 'a', encoding = 'utf-8') as f:
                f.write('\t' * (dept + 1) + str(subcategorie['pageid']) + '//' + subcategorie['title'] + '\n')
            get_all_content(subcategorie['title'], subcategorie['pageid'], dept+1, max_dept)

with open(r'D:\Huan\Project\KHDL\structure.txt', 'a', encoding = 'utf-8') as f:
    f.write('4155' + '//' + root_category + '\n')
get_all_content(root_category, 4155, 0, max_dept = 5)
    
        



Tải dữ liệu wiki dưới dạng pdf


In [4]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_title_from_page_id(page_id):
    url = "https://vi.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "pageids": page_id,
        "format": "json"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data["query"]["pages"][str(page_id)]["title"]
    else:
        print(f"Lỗi khi lấy title cho page_id {page_id}. Mã lỗi: {response.status_code}")
        return None

def download_pdf(title, output_file):
    pdf_url = f"https://vi.wikipedia.org/api/rest_v1/page/pdf/{title.replace(' ', '_').replace('/', '%2f')}"
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(output_file, "wb") as file:
            file.write(response.content)
        print(f"Tải xuống thành công: {output_file}")
    else:
        print(f"Lỗi tải xuống PDF cho {title}. Mã lỗi: {response.status_code}")

def download_wikipedia_pdfs(page_ids):
    with ThreadPoolExecutor(max_workers=2) as executor:  # Tải đồng thời với 10 luồng
        future_to_title = {}

        # Lấy title từ page_id
        for page_id in page_ids:
            title = get_title_from_page_id(page_id)
            if title:
                output_file = f"pdf/{page_id}-{title.replace(' ', '_').replace('/', '_')}.pdf"
                future = executor.submit(download_pdf, title, output_file)
                future_to_title[future] = title

        # Kiểm tra tiến trình tải
        for future in as_completed(future_to_title):
            title = future_to_title[future]
            try:
                future.result()
            except Exception as e:
                print(f"Lỗi khi tải {title}: {e}")

# Ví dụ sử dụng
import pandas as pd
page_ids = []
df = pd.read_csv('data.csv') 
for index, row in df.iterrows():
    page_ids.append(row['0'])
download_wikipedia_pdfs(page_ids)


Tải xuống thành công: pdf/13477390-Tự_thụ_phấn.pdf
Tải xuống thành công: pdf/13478635-Nhị_(thực_vật).pdf
Tải xuống thành công: pdf/140495-Hoa.pdf
Tải xuống thành công: pdf/15288019-Thụ_phấn_kín.pdf
Tải xuống thành công: pdf/15288208-Thụ_phấn_mở.pdf
Tải xuống thành công: pdf/15288394-Thụ_phấn_cùng_cây.pdf
Tải xuống thành công: pdf/1610333-Thụ_phấn_nhờ_gió.pdf
Tải xuống thành công: pdf/19496342-Cách_li_sinh_sản.pdf
Tải xuống thành công: pdf/19652468-Thành_viên:FoxSerfaty_Nháp_Hoa.pdf
Tải xuống thành công: pdf/19735262-Bao_phấn.pdf
Tải xuống thành công: pdf/19879395-Trụ_nhị_nhụy.pdf
Tải xuống thành công: pdf/32874-Thực_vật_có_hoa.pdf
Tải xuống thành công: pdf/415874-Quả.pdf
Tải xuống thành công: pdf/669775-Xenohyla_truncata.pdf
Tải xuống thành công: pdf/793924-Cánh_hoa.pdf
Tải xuống thành công: pdf/81333-Thụ_phấn.pdf
Tải xuống thành công: pdf/87782-Thụ_phấn_nhờ_côn_trùng.pdf
Tải xuống thành công: pdf/94691-Mẫu_lõi_phấn_hoa.pdf
Tải xuống thành công: pdf/95288-Phấn_hoa.pdf
Tải xuống thành c

KeyboardInterrupt: 

In [8]:
import pandas as pd

df = pd.read_json(r'ftdata\embed_data\test_embed.jsonl', encoding='utf-8', lines=True)

In [9]:
print(df)

                                                  query  \
0           Tá dược của Amaryl gồm những thành phần gì?   
1     Nội soi được thực hiện như thế nào trong quá t...   
2     Đau đầu do căng thẳng có biểu hiện và cách điề...   
3                           Xương mác có bao nhiêu đầu?   
4         Bệnh hen tim có thể dẫn đến suy hô hấp không?   
...                                                 ...   
8130         Cái ghẻ là loại ký sinh trùng như thế nào?   
8131  Giai đoạn đầu cần làm gì khi nghi ngờ mắc chứn...   
8132   Phải phẩu thuật khi các bướu giáp như sau không?   
8133                         Khoai nưa có tác dụng nào?   
8134  Nguyên nhân nào khác dẫn đến hình thành nang t...   

                                                    pos  \
0     [- Mỗi viên Amaryl 1 mg có chứa 1 mg glimepiri...   
1     [Bác sĩ dùng dụng cụ nôi soi (một ống camera n...   
2     [Một số tình trạng có thể làm xuất hiện các tr...   
3     [Cẳng chân được tạo thành bởi hai xương – xươn...

In [10]:
pair_score = []
for index, row in df.iterrows():
    for i in row['pos']:
        pair_score.append((row['query'], i, 1.0))
    for i in row['neg']:
        pair_score.append((row['query'], i, 0.0))
        


In [11]:
import csv

# Mở file CSV để ghi
with open('test_pair_score.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Ghi tiêu đề cột (nếu cần)
    writer.writerow(["sentence1", "sentence2", "score"])

    # Ghi dữ liệu
    writer.writerows(pair_score)

print("Dữ liệu đã được lưu vào file CSV.")


Dữ liệu đã được lưu vào file CSV.
